<a href="https://colab.research.google.com/github/0011Ashwin/Google-Colab-Work/blob/main/student_classroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ------------------------------------
# ✅ Install packages if needed
#!pip install -q ultralytics opencv-python

from ultralytics import YOLO
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from google.colab import files

# ------------------------------------
# ✅ Upload the Input Video

uploaded = files.upload()
for fn in uploaded.keys():
    video_path = '/content/' + fn
    print(f"✅ Uploaded file: {video_path}")

# ------------------------------------
# ✅ Open the Video

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise Exception("❌ Error opening input video. Please upload a correct video.")

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
duration = total_frames / fps if fps > 0 else 0

print(f"🎥 Video loaded: {total_frames} frames, {fps:.2f} FPS, {duration:.2f} seconds.")

if total_frames < 10:
    raise Exception("⚠️ Input video is too short! Upload a longer video (more than 2 seconds).")

# ------------------------------------
# ✅ Setup Output Video

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

output_path = '/content/classroom_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# ------------------------------------
# ✅ Load the YOLOv8 Pose Model

model = YOLO('yolov8n-pose.pt')  # 'n' is fast. Use 's', 'm', 'l' if you want better accuracy

# ------------------------------------
# ✅ Stand/Sit Classification Function

def classify_pose_stand_or_sit(keypoints):
    """
    Classify if student is 'Standing' or 'Sitting' based on keypoints (hips and shoulders).
    """
    if keypoints is None or len(keypoints) < 17:
        return 'Unknown'

    left_hip = keypoints[11][:2]
    right_hip = keypoints[12][:2]
    left_shoulder = keypoints[5][:2]
    right_shoulder = keypoints[6][:2]

    hip_mid = (left_hip + right_hip) / 2
    shoulder_mid = (left_shoulder + right_shoulder) / 2

    vertical_distance = hip_mid[1] - shoulder_mid[1]  # y-axis

    # Heuristic rule
    if vertical_distance > 100:
        return 'Standing'
    else:
        return 'Sitting'

# ------------------------------------
# ✅ Process the Video

frame_count = 0
saved_frames = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    frame_count += 1

    # Run pose detection
    results = model.predict(frame, imgsz=640, conf=0.4, verbose=False)

    annotated_frame = results[0].plot()

    # Process detected people
    for person in results[0].keypoints.xy:
        keypoints = person.cpu().numpy()

        # Add dummy confidence 1.0
        keypoints_full = np.concatenate([keypoints, np.ones((17,1))], axis=1)

        action = classify_pose_stand_or_sit(keypoints_full)

        if keypoints.shape[0] > 0:
            x, y = int(keypoints[0][0]), int(keypoints[0][1])
            cv2.putText(annotated_frame, action, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Save annotated frame
    out.write(annotated_frame)
    saved_frames += 1

    # Show frame every 10 frames
    if frame_count % 10 == 0:
        cv2_imshow(annotated_frame)
        clear_output(wait=True)

cap.release()
out.release()

print(f"✅ Detection Finished! Total frames processed: {saved_frames}")

# ------------------------------------
# ✅ Download the output video
files.download(output_path)


✅ Detection Finished! Total frames processed: 818


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>